In [208]:
import praw
import json
import re
from praw.models import MoreComments

In [209]:
file = open('reddit_info.json')
info = json.load(file)

In [210]:
#API Setup
def create_reddit_object():
    reddit = praw.Reddit(client_id = info[0]['client_id'],
                        client_secret = info[0]['client_secret'],
                        user_agent = info[0]['user_agent'],
                        #username & password is optional if you are only looking at public comments
                        username = info[0]['username'],
                        password = info[0]['password'])
    return reddit

In [333]:
#Names of the video games to look for
video_games = ['super mario', 'zelda', 'kirby', 'pokemon', 'metroid', 'sonic', 'mega man', 'animal crossing',
              'pikmin', 'fire emblem', 'f-zero', 'golden sun', 'donkey kong', 'advance wars', 'xenoblade chronicles',
              'ratchet and clank', 'jak and daxter', 'sly cooper', 'god of war', 'persona', 'final fantasy',
              'resident evil', 'silent hill', 'uncharted', 'halo', 'street fighter', 'mortal kombat', 'gears of war',
              'devil may cry', 'castlevania', 'crash bandicoot', 'spyro', 'metal gear', 'doom', 'bioshock', 'kingdom hearts',
              'fallout', 'monster hunter', 'gta', 'dark souls']

single_word_games = []
double_word_games = []
triple_word_games = []

space_count = 0
for game in video_games:
    for letter in game:
        if letter == ' ':
            space_count += 1
    if space_count == 0:
        single_word_games.append(game)
    elif space_count == 1:
        double_word_games.append(game)
    elif space_count == 2:
        triple_word_games.append(game)
    
    space_count = 0
        
print(single_word_games)
print(double_word_games)
print(triple_word_games)
        

#Setting up word finding
pattern = re.compile(r'(\w+ \w+)|(\w+)')
pattern_groups = [re.compile(r'super mario'),
                 re.compile(r'zelda')]
matches = pattern.finditer(comment.body)

['zelda', 'kirby', 'pokemon', 'metroid', 'sonic', 'pikmin', 'f-zero', 'persona', 'uncharted', 'halo', 'castlevania', 'spyro', 'doom', 'bioshock', 'fallout', 'gta']
['super mario', 'mega man', 'animal crossing', 'fire emblem', 'golden sun', 'donkey kong', 'advance wars', 'xenoblade chronicles', 'sly cooper', 'final fantasy', 'resident evil', 'silent hill', 'street fighter', 'mortal kombat', 'crash bandicoot', 'metal gear', 'kingdom hearts', 'monster hunter', 'dark souls']
['ratchet and clank', 'jak and daxter', 'god of war', 'gears of war', 'devil may cry']


In [214]:
#sets up reddit obj, url to go through
reddit = create_reddit_object()
url = "https://www.reddit.com/r/gaming/comments/obysq4/pokemon_gta_and_halo/"
submission = reddit.submission(url=url)
comments = []

#This next code takes ages to run. There's about 15,000 comments to gather
submission.comments.replace_more(limit=None)
comments = submission.comments.list()

print(len(comments))


15026


In [346]:
#Setting up dictionary
video_games_count = dict()
for game in video_games:
    video_games_count[game] = 1

#trying something

def match_confirm(match, comment):
    if match != None:
        span = list(match.span())
        idx1 = span[0]
        idx2 = span[1]
        return(comment[idx1:idx2])


for comment in comments:
    body = comment.body.lower()
    for i in range(len(single_word_games)):
        match = re.match(single_word_games[i], body)
        game = match_confirm(match, body)
        if isinstance(game, str):
            video_games_count[game] += 1 
            
    for i in range(len(double_word_games)):
        match = re.match(double_word_games[i], body)
        game = match_confirm(match, body)
        if isinstance(game, str):
            video_games_count[game] += 1 
            
    for i in range(len(triple_word_games)):
        match = re.match(triple_word_games[i], body)
        game = match_confirm(match, body)
        if isinstance(game, str):
            video_games_count[game] += 1 

print(video_games_count)

{'super mario': 474, 'zelda': 1409, 'kirby': 118, 'pokemon': 963, 'metroid': 224, 'sonic': 133, 'mega man': 53, 'animal crossing': 83, 'pikmin': 72, 'fire emblem': 118, 'f-zero': 29, 'golden sun': 148, 'donkey kong': 45, 'advance wars': 44, 'xenoblade chronicles': 22, 'ratchet and clank': 122, 'jak and daxter': 65, 'sly cooper': 124, 'god of war': 213, 'persona': 192, 'final fantasy': 490, 'resident evil': 174, 'silent hill': 64, 'uncharted': 98, 'halo': 747, 'street fighter': 65, 'mortal kombat': 106, 'gears of war': 50, 'devil may cry': 49, 'castlevania': 55, 'crash bandicoot': 46, 'spyro': 95, 'metal gear': 213, 'doom': 376, 'bioshock': 224, 'kingdom hearts': 167, 'fallout': 612, 'monster hunter': 263, 'gta': 530, 'dark souls': 730}


In [350]:
#Sorts dictionary
sorted_dict = {}
sorted_keys = sorted(video_games_count)  # [1, 3, 2]

video_games_count = dict(sorted(video_games_count.items(), key=lambda item: item[1], reverse=True))

game_count = 0
for key in video_games_count:
    game_count += video_games_count[key]


print('Total comments scanned: ', len(comments))
print('Total video games found: ', game_count, end='\n\n')

for key in video_games_count:
    print(f'{key} : {video_games_count[key]}')

Total comments scanned:  15026
Total video games found:  9805

zelda : 1409
pokemon : 963
halo : 747
dark souls : 730
fallout : 612
gta : 530
final fantasy : 490
super mario : 474
doom : 376
monster hunter : 263
metroid : 224
bioshock : 224
god of war : 213
metal gear : 213
persona : 192
resident evil : 174
kingdom hearts : 167
golden sun : 148
sonic : 133
sly cooper : 124
ratchet and clank : 122
kirby : 118
fire emblem : 118
mortal kombat : 106
uncharted : 98
spyro : 95
animal crossing : 83
pikmin : 72
jak and daxter : 65
street fighter : 65
silent hill : 64
castlevania : 55
mega man : 53
gears of war : 50
devil may cry : 49
crash bandicoot : 46
donkey kong : 45
advance wars : 44
f-zero : 29
xenoblade chronicles : 22
